# Interactive Layer Clipping

## Step 1: Define your AOI

Draw a rectangle on the map using the rectangle control. This will be used as the bounding box of the clipped raster.

## Step 2: Define your target SRS

Provide your target SRS as an EPSG code.  The identified name of the EPSG code is provided for your reference.

You must also provide a cell size in your target coordinate system's units.

## Step 2: Click the button to start clipping

Logging will be printed below the map.

In [ ]:
import os
import logging
import sys
from urllib.parse import parse_qs

import ipywidgets
from IPython import get_ipython
import IPython.display

LOGGER = logging.getLogger(__name__)

# Check the query parameters if we're in voila
if (os.environ.get('SERVER_SOFTWARE', False) and 
        os.environ['SERVER_SOFTWARE'].startswith('voila')):      
    # localhost:8866?foo=bar,baz will return {'foo': ['bar', 'baz']}
    query_string = os.environ.get('QUERY_STRING', '')
    parameters = parse_qs(query_string)
    display("query string parameters:", parameters)
else:
    # We're not running in voila, default to no parameters.
    parameters = {}

In [ ]:
from ipyleaflet import basemap_to_tiles, basemaps
import ipywidgets

import dataportal_widgets
import output_options
import interactive_options

output_handler = output_options.OutputWidgetHandler()
logging.getLogger().addHandler(output_handler)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('pygeoprocessing').setLevel(logging.DEBUG)

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

if 'initial_layer' in parameters:
    initial_layer = parameters['initial_layer']
else:
    # just assume SRTM if the user hasn't specified a layer.
    initial_layer = "srtm-v3-1s"

m = dataportal_widgets.DataportalMap(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
    initial_layer=initial_layer
)

epsg_input = output_options.EPSG_INPUT
epsg_label = output_options.EPSG_LABEL
pixel_sizes_label = output_options.PIXEL_SIZES_LABEL


def _detect_srs_name(change):
    try:
        if isinstance(change['new'], str):
            epsg_code = int(change['new'])
        elif isinstance(change['new'], dict):
            epsg_code = int(change['new']['value'])
        else:
            raise Exception(change)
    except (ValueError, TypeError):
        epsg_code = None
    except KeyError:
        # no new value was set; skip
        return
        
    label, pixel_size_text = output_options.parse_epsg_code(epsg_code)
        
    epsg_label.value = label
    pixel_sizes_label.value = pixel_size_text
    
epsg_input.observe(_detect_srs_name)

pixel_size_x_input = output_options.PIXEL_SIZE_X_INPUT
pixel_size_y_input = output_options.PIXEL_SIZE_Y_INPUT 

output_params_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Output options</b>"),
    ipywidgets.HBox([epsg_input, epsg_label]),
    ipywidgets.HBox(
        [pixel_size_x_input, pixel_size_y_input, pixel_sizes_label]),
])

logging_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Logging</b>"),
    output_handler.out,
])

submit = ipywidgets.Button(
    description='Clip raster',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clip the selected raster by the selected bounding box',
    icon='crop' # (FontAwesome names without the `fa-` prefix)
)
download_link = ipywidgets.HTML()
submit_hbox = ipywidgets.HBox([submit, download_link])
logging_box_is_displayed=False
def _submit(button):
    button.disabled=True
    try:
        print(button)
        if not logging_box_is_displayed:
            display(logging_box)
        dataset_key = m.selected_map()
        target_path = 'foo.tif'
        interactive_options.compute(
            f"/vsicurl/{dataportal_widgets.get_url(m.selected_map())}",
            m.selected_bbox(),
            epsg_input.value,
            target_path,
            (float(pixel_size_x_input.value), float(pixel_size_y_input.value)),
        )
    finally:
        # always re-enable the button afterwards
        button.disabled=False
        download_link.value = f"<a href={target_path} download>Download {os.path.basename(target_path)}</a>" 
    
submit.on_click(_submit)

display(m)
display(output_params_box)
display(submit_hbox)
#display(logging_box)


In [ ]:
#a = m.selected_bbox()
#(ulx, uly), (llx, lly) = a
#import shapely.geometry
#shapely.geometry.shape(shapely.geometry.box(ulx, llx, uly, lly))
#m.selected_map()
print(URL)

In [ ]:
# build a colormap from our GDAL-compatible GRASS color file
import json
import requests

from osgeo import gdal
import matplotlib
import branca
gdal.DontUseExceptions()

DATASETS_BASE_URL = "https://storage.googleapis.com/gef-ckan-public-data"
DATASETS_DIRECTORY_URL = f"{DATASETS_BASE_URL}/directory.json"
DATASET_DIRECTORY = requests.get(DATASETS_DIRECTORY_URL).json()['public_layers']


def _load_colormap(dataset_key):
    assert dataset_key in DATASET_DIRECTORY, f"Unknown dataset key {dataset_key}"
    dataset_info = DATASET_DIRECTORY[dataset_key]
    raster_path = f"/vsicurl/{DATASETS_BASE_URL}/{dataset_key}/{dataset_info['download']}"
    
    try:
        raster_info = gdal.Info(raster_path, format="json")
        minimum = raster_info['bands'][0]['min']
        maximum = raster_info['bands'][0]['max']
    except (TypeError, KeyError):
        # TypeError when raster can't be found (info is None)
        # KeyError when min/max not defined
        LOGGER.exception(f"Could not load min/max from {raster_path}")
        minimum = -1000
        maximum = 1000

    
    colors_filename = dataset_info['colors']
    colors_url = f"{DATASETS_BASE_URL}/{dataset_key}/{colors_filename}"
    
    colormap_list = []
    for line in requests.get(colors_url).iter_lines():
        # Requires that we have ASCII text only.
        # GRASS color files allow ":" as a color separator, so replace with space for ease of parsing.
        line = line.decode('ASCII').replace(':', ' ')
        if line.startswith('#'):
            continue
        percent, colors = line.split(' ', 1)  # Stop after the first split
        
        colors = [c for c in colors.split(' ') if c]
        if len(colors) == 1:
            r, g, b = matplotlib.colors.to_rgb(colors[0])
        elif len(colors) == 3:
            r, g, b = colors
        else:
            r, g, b, a = colors

        colormap_list.append((float(r), float(g), float(b)))
        

    colormap = branca.colormap.LinearColormap(
        colors=colormap_list,
        vmin=minimum, vmax=maximum)
    colormap.caption = 'Elevation (m)'  # TODO: read this in from metadata
    return f"{colormap._repr_html_()}"

colormap = _load_colormap('awc-isric-soilgrids')

In [ ]:
import json
import threading
import logging


import pygeoprocessing
import shapely.geometry
from osgeo import gdal, osr
import ipyleaflet
from ipyleaflet import Map, LocalTileLayer, basemap_to_tiles, basemaps, Rectangle, TileLayer
from ipyleaflet import LayersControl, ZoomControl, ScaleControl, LegendControl, WidgetControl, DrawControl
import IPython.display
import ipywidgets as widgets

import output_options
import interactive_options

        
output_handler = output_options.OutputWidgetHandler()
output_handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logging.getLogger().addHandler(output_handler)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('pygeoprocessing').setLevel(logging.DEBUG)

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'
m = Map(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
)

scalebar = ScaleControl(position='bottomleft')
m.add_control(scalebar)

# If we want to pre-cache these tiles on GCS, use ipyleaflet.leaflet.TileLayer instead
country_id_tiles_layer = LocalTileLayer(
    path='tiles/{z}/{x}/{y}.png',
    name='NASA SRTM',
    show_loading=True,
    attribution='NASA'
    
)
m.add_layer(country_id_tiles_layer)
tiles_layers = [country_id_tiles_layer]

colorbar_html = widgets.HTML()
colorbar = WidgetControl(widget=colorbar_html, position='topright')
m.add_control(colorbar)

options = []
for available_layer_name, layer_data in DATASET_DIRECTORY.items():
    options.append(
        (layer_data['title'], available_layer_name)
    )
    
layers_dropdown = widgets.Dropdown(
    options=options,
    description='Preview layer'
)
layers_control = WidgetControl(widget=layers_dropdown, position='topright')
m.add_control(layers_control)


def _change_tile_layer(change):
    m.remove_layer(tiles_layers[0])
    tiles_layers.append(
        LocalTileLayer(
            path='tiles/{z}/{x}/{y}.png',
            name=DATASET_DIRECTORY[change['new']]['title'],
            show_loading=True,
            attribution='TODO'
        )
    )
    m.add_layer(tiles_layers[0])
    new_colors = _load_colormap(change['new'])
    colorbar_html.value = new_colors
    

layers_dropdown.observe(_change_tile_layer, 'value')


draw_control = DrawControl(polyline={}, circlemarker={})
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.5
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Whoops!"
    },
    "allowIntersection": False
}
m.add_control(draw_control)

submit_button = widgets.Button(
    description = 'Clip layer to AOI',
    disabled=False,  # start out disabled, enable when geometries defined
    button_style='',  # change to 'success' when geometries added
    tooltip='Clip the underlying layer by the bounding box of the geometries defined.',
    icon='scissors',  # this is a fontawesome icon
)
submit_button_control = WidgetControl(widget=submit_button, position='bottomright')


def _add_submit_button(control, **kwargs):
    # enable the submit button if 
    if kwargs['geo_json']['geometry']['coordinates']:
        m.add_control(submit_button_control)
    else:
        try:
            m.remove_control(submit_button_control)
        except ipyleaflet.ControlException:
            # When the control is not currently on the map
            pass

bbox_layers = []
def _draw_bbox(control, **kwargs):
    if bbox_layers:
        m.remove_layer(bbox_layers[0])
    if kwargs['geo_json']['geometry']['coordinates']:
        bbox_layer = interactive_options.leaflet_rectangle_from_bbox(
            kwargs['geo_json']['geometry'])
        bbox_layers.append(bbox_layer)
        m.add_layer(bbox_layer)
        
draw_control.on_draw(_add_submit_button)
draw_control.on_draw(_draw_bbox)


def _submit_form(*args, **kwargs):
    LOGGER.info('foo')
    submit_button.description = "Clipping ..."
    submit_button.disabled = True
    interactive_options.compute(
        'intermediate/global_dem.tif',
        draw_control.data[0]['geometry'],
        int(epsg_input.value),
        'foo.tif',
        (float(pixel_size_x_input.value), float(pixel_size_y_input.value)),
    )
    submit_button.disabled = False
    submit_button.description = 'Clip layer to AOI'
    
submit_button.on_click(_submit_form)

epsg_input = output_options.EPSG_INPUT
epsg_label = output_options.EPSG_LABEL
pixel_sizes_label = output_options.PIXEL_SIZES_LABEL


def _detect_srs_name(change):
    try:
        if isinstance(change['new'], str):
            epsg_code = int(change['new'])
        elif isinstance(change['new'], dict):
            epsg_code = int(change['new']['value'])
        else:
            raise Exception(change)
    except (ValueError, TypeError):
        epsg_code = None
    except KeyError:
        # no new value was set; skip
        return
        
    label, pixel_size_text = output_options.parse_epsg_code(epsg_code)
        
    epsg_label.value = label
    pixel_sizes_label.value = pixel_size_text
    
epsg_input.observe(_detect_srs_name)

pixel_size_x_input = output_options.PIXEL_SIZE_X_INPUT
pixel_size_y_input = output_options.PIXEL_SIZE_Y_INPUT 

output_params_box = widgets.VBox([
    widgets.HTML("<b>Output options</b>"),
    widgets.HBox([epsg_input, epsg_label]),
    widgets.HBox(
        [pixel_size_x_input, pixel_size_y_input, pixel_sizes_label]),
])

logging_box = widgets.VBox([
    widgets.HTML("<b>Logging</b>"),
    output_handler.out,
])

display(m)
display(output_params_box)
display(logging_box)

In [ ]:
# Remaining TODOs for this application:
# - [ ] expand this to operate on various layers available
# - [ ] expand this to write out rasters to a new session (grouped by uuid4()) on a bucket (maybe GDAL can directly write out to the bucket?)
# - [x] get the URL when running in Voila mode
# - [x] show the bounding box of the selected AOI polygon
# - [x] Allow the user to define their target EPSG code
# - [x] Use the user's target EPSG code in warping
# - [x] If the EPSG code represents a projected coord system (!osr.IsGeographic()), provide the pixel size
# - [x] Use osr.SpatialReference().GetName() to display the name of the coordinate system for user reference (unknown if unknown)